# 술 추천 알고리즘

도수 유사도는 `euclidean distance`로 구했고, 풍미 및 기타 정보의 유사도는 `cosine similarity`로 구했다.  
술의 고유 id를 받아 비슷한 술의 고유 id를 return한다.

### general_recommendation():
도수 유사도 50% 풍미(4가지) 유사도 50% 고려해 주종 상관 없이 비슷한 술을 찾아줌

### wine_recommendation():
도수 유사도 50% 기타(category, origin, producer, wine_grape, flavour) 50% 고려해 같은 대분류 내에서 비슷한 와인을 찾아줌

### the used variables of each alcohols

- 와인: 주종, 중분류, 주류명, 생산국가, 생산자/제조사, 주요품종, 도수, 음용온도, 추천음식, 맛
- 막걸리 : 주종, 주류명, 생산국가, 생산자/제조사, 주원료, 용량, 도수  
- 브랜디 : '주종','주류명','생산국가','용량','도수','특징'
- 위스키 : '주종','주류명','생산국가','용량','도수','특징'
- 국내주류 : '주종','주류명','생산국가','생산자/제조사','용량','도수','특징'

In [1]:
import numpy as np
import pandas as pd 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance_matrix
from scipy.spatial.distance import squareform
from sklearn.preprocessing import MinMaxScaler
from string import punctuation
import re


In [22]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       400 non-null    int64  
 1   주종       400 non-null    object 
 2   중분류      400 non-null    object 
 3   상품명      400 non-null    object 
 4   생산국가     400 non-null    object 
 5   생산자      400 non-null    object 
 6   주요품종     400 non-null    object 
 7   도수       400 non-null    float64
 8   음용온도     400 non-null    object 
 9   추천음식     399 non-null    object 
 10  당도       400 non-null    int64  
 11  바디       400 non-null    int64  
 12  산도       400 non-null    int64  
 13  타닌       400 non-null    int64  
 14  flavour  400 non-null    object 
dtypes: float64(1), int64(5), object(9)
memory usage: 47.0+ KB


In [2]:
wine = pd.read_csv("wine400.csv")
wine

,id,주종,중분류,상품명,생산국가,생산자,주요품종,도수,음용온도,추천음식,당도,바디,산도,타닌
0,1,wine,레드,"7 컬러즈, '그랑 리제르바' 카베르네 소비뇽 뮈스카",칠레,세븐 컬러즈 7 Colores,"카베르네 소비뇽 (Cabernet Sauvignon) 95%, 뮈스까 (Muscat...",13.5,16~18 ℃,"채끝 스테이크, 크림 파스타, 블루치즈 버거 등과 잘 어울린다.",1,3,3,3
1,2,wine,레드,"GCF, 꼬뜨 드 뵈프",프랑스,GCF 그룹 GCF Group,"시라/쉬라즈 (Syrah/Shiraz) , 마르셀란 (Marselan)",13.0,16~18 ℃,"뵈프 부르기뇽, 안심 찹 스테이크, 로스트 비프 스테이크 등과 잘 어울린다.",1,3,3,2
2,3,wine,화이트,"가트, 리슬링",호주,가트 와인즈 Gatt Wines,리슬링 (Riesling) 100%,11.5,10~12 ℃,"샐러드, 해산물, 치즈 등과 잘 어울린다.",2,4,3,1
3,4,wine,레드,갈로 리빙스톤 콩코드,미국,갤로 패밀리 빈야드 Gallo Family Vineyard - E&J Gallo W...,콩코드 (Concord),11.0,14~16 ℃,"소시지, 육포, 떡볶이, 순대, 튀김, 족발, 만두, 자장면 등과 잘 어울린다.",1,4,3,3
4,5,wine,주정강화,글로리아 토니 포트 와인,포르투갈,빈센테 파리아 비뉴스 Vincente Faria VInhos,"틴타 로리즈 (Tinta Roriz) , 투리가 프란카 (Touriga Franca...",20.0,14~16 ℃,"리코타, 크림치즈, 말린과일, 케이크, 초콜렛 등 디저트와 잘 어울린다.",4,2,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,396,wine,레드,"매티스, 그르나슈",미국,매티스 와이너리 Mathis Winery,"그르나슈 (Grenache) 82%, 까리냥 (Carignan/Carignane) ...",14.0,16~18 ℃,"덩어리 째 오븐에 구운 육류와 햄, 스테이크, 라자냐, 치즈 등과 잘 어울린다.",1,4,2,1
396,397,wine,화이트,"메종 카스텔, 소비뇽 블랑",프랑스,메종 카스텔 Maison Castel,소비뇽 블랑 (Sauvignon Blanc),13.0,8~10 ℃,"연어 타르타르 스테이크와 같은 해산물 요리, 샐러드와 같은 야채, 생선 그릴 요리와...",1,3,4,5
397,398,wine,스파클링,"메종 카스텔, 퀴베 블랑쉬 브뤼",프랑스,메종 카스텔 Maison Castel,샤르도네 (Chardonnay),11.0,6~8 ℃,"디저트, 조개, 생선 요리 또는 반주와 잘 어울린다.",1,4,1,1
398,399,wine,화이트,"메종 카스텔, 푸이 퓌메",프랑스,메종 카스텔 Maison Castel,소비뇽 블랑 (Sauvignon Blanc),13.0,8~10 ℃,"샤프란을 곁들인 생선 스튜 요리 등 생선, 갑각류, 조개 요리, 크림 소스를 곁들인...",1,4,1,1


In [15]:
# 도수 유사도 구하기
percent = wine['도수']
dist_pair = []

# y축에 임의로 0을 부여한 거리 순서쌍 생성
for i in range(0,len(percent)):
    temp = []
    temp.append(wine.loc[i]['도수'])
    temp.append(0)
    dist_pair.append(temp)

# get a distance matrix
df = pd.DataFrame(dist_pair, columns=['x', 'y'])
dist_matrix = distance_matrix(df.values, df.values)

# 정규화
min_max_scaler = MinMaxScaler()
regularised = min_max_scaler.fit_transform(dist_matrix)

# 1에서 빼줘서 더 가까운 것이 우선순위를 갖도록 변경하기
one_matrix = np.ones((400,400))

final_dist = one_matrix - regularised

# 확인
print(final_dist)

[[1.         0.95       0.76470588 ... 0.72222222 0.95       0.68421053]
 [0.95238095 1.         0.82352941 ... 0.77777778 1.         0.73684211]
 [0.80952381 0.85       1.         ... 0.94444444 0.85       0.89473684]
 ...
 [0.76190476 0.8        0.94117647 ... 1.         0.8        0.94736842]
 [0.95238095 1.         0.82352941 ... 0.77777778 1.         0.73684211]
 [0.71428571 0.75       0.88235294 ... 0.94444444 0.75       1.        ]]


In [5]:
# flavour word list 만들기
flavour_list = [] # empty list

for i in range(0, len(wine)):
    temp = ""
    
    if wine.loc[i]['당도'] >= 3:
        temp = temp + "당도 "
    if wine.loc[i]['바디'] >= 3:
        temp = temp + "바디 "
    if wine.loc[i]['산도'] >= 3:
        temp = temp + "산도 "
    if wine.loc[i]['타닌'] >= 3:
        temp = temp + "타닌 "
    
    flavour_list.append(temp)

# 해당 리스트 데이터 프레임에 추가
wine["flavour"] = pd.DataFrame({"flavour":flavour_list})
flavour = wine['flavour']

# flavour 코사인 유사도 구하기
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(flavour)

# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

# 확인
print(cosine_sim)

[[1.         0.81649658 0.81649658 ... 0.57735027 0.57735027 0.57735027]
 [0.81649658 1.         1.         ... 0.70710678 0.70710678 0.70710678]
 [0.81649658 1.         1.         ... 0.70710678 0.70710678 0.70710678]
 ...
 [0.57735027 0.70710678 0.70710678 ... 1.         1.         1.        ]
 [0.57735027 0.70710678 0.70710678 ... 1.         1.         1.        ]
 [0.57735027 0.70710678 0.70710678 ... 1.         1.         1.        ]]


In [6]:
wine

,id,주종,중분류,상품명,생산국가,생산자,주요품종,도수,음용온도,추천음식,당도,바디,산도,타닌,flavour
0,1,wine,레드,"7 컬러즈, '그랑 리제르바' 카베르네 소비뇽 뮈스카",칠레,세븐 컬러즈 7 Colores,"카베르네 소비뇽 (Cabernet Sauvignon) 95%, 뮈스까 (Muscat...",13.5,16~18 ℃,"채끝 스테이크, 크림 파스타, 블루치즈 버거 등과 잘 어울린다.",1,3,3,3,바디 산도 타닌
1,2,wine,레드,"GCF, 꼬뜨 드 뵈프",프랑스,GCF 그룹 GCF Group,"시라/쉬라즈 (Syrah/Shiraz) , 마르셀란 (Marselan)",13.0,16~18 ℃,"뵈프 부르기뇽, 안심 찹 스테이크, 로스트 비프 스테이크 등과 잘 어울린다.",1,3,3,2,바디 산도
2,3,wine,화이트,"가트, 리슬링",호주,가트 와인즈 Gatt Wines,리슬링 (Riesling) 100%,11.5,10~12 ℃,"샐러드, 해산물, 치즈 등과 잘 어울린다.",2,4,3,1,바디 산도
3,4,wine,레드,갈로 리빙스톤 콩코드,미국,갤로 패밀리 빈야드 Gallo Family Vineyard - E&J Gallo W...,콩코드 (Concord),11.0,14~16 ℃,"소시지, 육포, 떡볶이, 순대, 튀김, 족발, 만두, 자장면 등과 잘 어울린다.",1,4,3,3,바디 산도 타닌
4,5,wine,주정강화,글로리아 토니 포트 와인,포르투갈,빈센테 파리아 비뉴스 Vincente Faria VInhos,"틴타 로리즈 (Tinta Roriz) , 투리가 프란카 (Touriga Franca...",20.0,14~16 ℃,"리코타, 크림치즈, 말린과일, 케이크, 초콜렛 등 디저트와 잘 어울린다.",4,2,4,3,당도 산도 타닌
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,396,wine,레드,"매티스, 그르나슈",미국,매티스 와이너리 Mathis Winery,"그르나슈 (Grenache) 82%, 까리냥 (Carignan/Carignane) ...",14.0,16~18 ℃,"덩어리 째 오븐에 구운 육류와 햄, 스테이크, 라자냐, 치즈 등과 잘 어울린다.",1,4,2,1,바디
396,397,wine,화이트,"메종 카스텔, 소비뇽 블랑",프랑스,메종 카스텔 Maison Castel,소비뇽 블랑 (Sauvignon Blanc),13.0,8~10 ℃,"연어 타르타르 스테이크와 같은 해산물 요리, 샐러드와 같은 야채, 생선 그릴 요리와...",1,3,4,5,바디 산도 타닌
397,398,wine,스파클링,"메종 카스텔, 퀴베 블랑쉬 브뤼",프랑스,메종 카스텔 Maison Castel,샤르도네 (Chardonnay),11.0,6~8 ℃,"디저트, 조개, 생선 요리 또는 반주와 잘 어울린다.",1,4,1,1,바디
398,399,wine,화이트,"메종 카스텔, 푸이 퓌메",프랑스,메종 카스텔 Maison Castel,소비뇽 블랑 (Sauvignon Blanc),13.0,8~10 ℃,"샤프란을 곁들인 생선 스튜 요리 등 생선, 갑각류, 조개 요리, 크림 소스를 곁들인...",1,4,1,1,바디


In [7]:
# 도수, flavour를 모두 고려한 similarity 구하기 (weight는 각각 0.5)
new_sim = 0.5 * cosine_sim + 0.5 * final_dist

print(new_sim)

[[1.         0.88324829 0.79060123 ... 0.64978625 0.76367513 0.6307804 ]
 [0.88443877 1.         0.91176471 ... 0.74244228 0.85355339 0.72197444]
 [0.8130102  0.925      1.         ... 0.82577561 0.77855339 0.80092181]
 ...
 [0.66962752 0.75355339 0.82414163 ... 1.         0.9        0.97368421]
 [0.76486561 0.85355339 0.7653181  ... 0.88888889 1.         0.86842105]
 [0.64581799 0.72855339 0.79472986 ... 0.97222222 0.875      1.        ]]


In [8]:
# 여러 주종 내 추천
# 주종에 상관 없이 도수와 풍미만 고려해 비슷한 술을 추천해줌
# 항목: 도수, 풍미 (약 6개 항목)

# 고유 id를 넣으면 해당 술과 비슷한 Top 10의 id를 return
def general_recommendation(input_id, new_sim = new_sim):
    
    # 고유 id로 index 찾기 
    idx = wine.index[wine['id'] == input_id].tolist() # Int64Index 형식이라 list로 바꾸어줌
    
    # 해당 index의 유사도 리스트 sort in descending order
    score_series = pd.Series(new_sim[idx[0]]).sort_values(ascending = False)
    
    # 유사도 Top 10의 index 추출
    top_10_indexes = list(score_series.iloc[1:11].index)

    # 유사도 1인 항목이 하나 더 있어서 자기 자신이 포함되는 경우에는 자신을 뺀 Top 10의 index 재추출
    if top_10_indexes[0] == idx[0]:
        top_10_indexes = list(score_series.iloc[1:12].index)
        top_10_indexes.remove(idx[0])
    
    # 고유 id를 담기 위한 empty list 생성
    top_10_id = []
    
    # id list
    for i in top_10_indexes:
        id = wine.loc[i]['상품명']
        top_10_id.append(id)
    
    return top_10_id

# test id = 10 
print(general_recommendation(10))

['샤를 오르방, 뀌베 스페셜 08', '루이 로드레, 브뤼 프리미에', '트레드 소프틀리, 로제', '노비볼레 로마냐 스푸만테 비앙코', '와비사비, 726 바인스 바하우', '루이 로드레, 크리스탈', '뻬레 벤뚜라, 까바 브륏 레세르바 아트 에디션 화이트', '빌라 산디, 프로세코 뀌베 오리스', '가트, 리슬링', '도츠, 아모르 드 도츠 로제']


In [19]:
# wine word list 만들기
wine_list = [] # empty list

# wine index list 추출
wine_idx = wine.index[wine['주종'] == "wine"].tolist()

# wine: category, percent, origin, producer, wine_grape, flavour
# 중분류, 도수, 생산자, 생산국가, 주요품종 

for i in wine_idx:
    temp = ""
    temp = temp + wine.loc[i]['중분류'] + " " + wine.loc[i]['생산국가'] + " " + wine.loc[i]['생산자'].replace(" ", "") + " " + wine.loc[i]['주요품종'].replace(" ", "") + " "
    
    for flavour in wine.loc[i]['flavour']:
        temp = temp + flavour

    wine_list.append(temp)

# wine 코사인 유사도 구하기
# instantiating and generating the count matrix
count = CountVectorizer()
wine_matrix = count.fit_transform(wine_list)

# generating the cosine similarity matrix
wine_sim = cosine_similarity(wine_matrix, wine_matrix)

# wine 도수 유사도 구하기
wine_pair = []

# y축을 임의로 0을 부여한 거리 순서쌍 생성
for i in wine_idx:
    temp = []
    temp.append(wine.loc[i]['도수'])
    temp.append(0)
    wine_pair.append(temp)

# get a distance matrix
wine_df = pd.DataFrame(wine_pair, columns=['x', 'y'])
wine_matrix = distance_matrix(wine_df.values, wine_df.values)

# 정규화
min_max_scaler = MinMaxScaler()
wine_regularised = min_max_scaler.fit_transform(wine_matrix)

# 1에서 빼줘서 더 가까운 것이 우선순위를 갖도록 변경하기
wine_one_matrix = np.ones((len(wine_idx),len(wine_idx)))

wine_final_dist = wine_one_matrix - wine_regularised

# 도수 유사도, 타 정보 유사도 각각 0.5씩 weight 부여 후 새로운 matrix 생성
wine_new_sim = 0.5 * wine_sim + 0.5 * wine_final_dist

# 확인
print(wine_new_sim)

[[1.         0.61136364 0.4889533  ... 0.42265686 0.53654575 0.39908555]
 [0.61255411 1.         0.51836506 ... 0.51198038 0.62309149 0.42540134]
 [0.51136226 0.53160036 1.         ... 0.54439101 0.56933757 0.78144497]
 ...
 [0.44249813 0.52309149 0.54275702 ... 1.         0.65       0.55083589]
 [0.53773622 0.62309149 0.55610227 ... 0.63888889 1.         0.5227244 ]
 [0.41412315 0.43198029 0.77525302 ... 0.5493739  0.52930335 1.        ]]


In [20]:
# 해당 id의 술이 wine인지 체크
def is_wine(input_id):
    temp_idx = wine.index[wine['id'] == input_id].tolist() # Int64Index 형식이라 list로 바꾸어줌
    result = wine.loc[temp_idx[0]]['주종'] == "wine"
    return result

# 와인의 고유 id를 넣으면 해당 와인과 비슷한 Top 10 와인의 id를 return
def wine_recommendation(input_id, wine_new_sim = wine_new_sim):
    
    # 고유 id로 index 찾기 
    idx = wine.index[wine['id'] == input_id].tolist() # Int64Index 형식이라 list로 바꾸어줌
    
    # wine_idx list 내에서 몇번째 와인인지 구하기
    w_idx = wine_idx.index(idx[0])
    
    # 해당 index의 유사도 리스트 sort in descending order
    score_series = pd.Series(wine_new_sim[w_idx]).sort_values(ascending = False)
    
    # 유사도 Top 10의 index 추출
    wine_top_10_indexes = list(score_series.iloc[1:11].index)

    # 유사도 1인 항목이 하나 더 있어서 자기 자신이 포함되는 경우에는 자신을 뺀 Top 10의 index 재추출
    if wine_top_10_indexes[0] == w_idx:
        wine_top_10_indexes = list(score_series.iloc[1:12].index)
        wine_top_10_indexes.remove(w_idx)
    
    # 고유 id를 담기 위한 empty list 생성
    wine_top_10_id = []
    
    # id list
    for i in wine_top_10_indexes:
        index = wine_idx[i] # wine list에서 몇번째인지가 아니라 전체 술 list에서 몇번째인지 구함
        id = wine.loc[index]['상품명']
        wine_top_10_id.append(id)
    
    return wine_top_10_id

In [21]:
# test
# 테스트로 임의의 아이디 넣음
test_num = 3

print(general_recommendation(test_num)) 

print(is_wine(test_num)) # 해당 id의 술이 wine인지 체크

print(wine_recommendation(test_num)) # 와인의 고유 id를 넣으면 해당 와인과 비슷한 Top 10 와인의 id를 return

['빌라 산디, 프로세코 뀌베 오리스', '뻬레 벤뚜라, 까바 브륏 레세르바 아트 에디션 화이트', '돔 페리뇽 P2', '샤를 오르방, 뀌베 스페셜 08', '엘 비에호 델 발레, 피노누아', '베리 브라더스 & 러드, 프로세코', '슐로스 폴라즈, 괴테 스페셜 에디션', '떼땅져, 빈티지 브뤼', '크룩, 빈티지 브뤼', '크룩, 컬렉션']
True
['슐로스 폴라즈, 괴테 스페셜 에디션', '슐로스 폴라즈, 에디션', '미션, 리슬링', '닥터 파우스트, 리슬링 스페트레제', '멜쉐이머, 리슬링 파인헤릅', '와비사비, 726 바인스 바하우', '이모스, 리슬링', '슐로스 고벨스버그, 리슬링 가이스버그', '도멘 오즈테르타그, 리슬링 레 쟈뎅', '도멘 갸데, 레 뻬리에르 모노폴']
